In [1]:
from spacy.lang.id import Indonesian
!pip install spacy python-crfsuite unidecode textblob sastrawi
import nltk
nltk.download('popular')
from nltk.tag import CRFTagger
import re
import pandas as pd
from nltk.corpus import stopwords
stop = set(stopwords.words('indonesian'))
import copy
import csv
import re

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/jackieleonardy/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/jackieleonardy/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/jackieleonardy/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/jackieleonardy/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/jackieleonardy/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/jackieleonardy/nltk_data...
[nl

In [2]:
nlp_id = Indonesian()

## Importing and reading File CSV Method

In [3]:
class CSVLoader():
    def __init__(self, csvFileName, newLine = ''):
        self.csvFileName = csvFileName
        self.newLine = newLine
        self.provinsiArray = ""
    
    def __openFile(self):
        with open(self.csvFileName, newline='') as f:
            reader = csv.reader(f)
            self.provinsiArray = list(reader)
            
    def getCSVArray(self):
        self.__openFile()
        return self.provinsiArray



In [4]:
provinsi_dictionary = CSVLoader("listProvinsi.csv").getCSVArray()
kabupaten_dictionary = CSVLoader("newlistKabupaten.csv").getCSVArray()
animal_dictionary = CSVLoader("Animal_Dictionary.csv").getCSVArray()

print(provinsi_dictionary)

[['aceh'], ['sumatera utara'], ['sumatera barat'], ['riau'], ['jambi'], ['sumatera selatan'], ['bengkulu'], ['lampung'], ['kepulauan bangka belitung'], ['kepulauan riau'], ['dki jakarta'], ['jawa barat'], ['jawa tengah'], ['di yogyakarta'], ['jawa timur'], ['banten'], ['bali'], ['nusa tenggara barat'], ['nusa tenggara timur'], ['kalimantan barat'], ['kalimantan tengah'], ['kalimantan selatan'], ['kalimantan timur'], ['kalimantan utara'], ['sulawesi utara'], ['sulawesi tengah'], ['sulawesi selatan'], ['sulawesi tenggara'], ['gorontalo'], ['sulawesi barat'], ['maluku'], ['maluku utara'], ['papua barat'], ['papua'], ['jakarta'], ['yogyakarta'], ['jawa']]


## Processing Corpus

In [5]:
class DictionaryProcessor():

    def __init__(self, inputDictionary = ""):
        self.dictionary = inputDictionary
    
    def processDictionary(self):
        self.__chunkDictionary()
        self.__splitDictionary()
    
    def __splitDictionary(self):
        dictionary_index = 0
        for dictionaries in self.dictionary:
            self.dictionary[dictionary_index] = dictionaries.split()
            dictionary_index += 1

    def __chunkDictionary(self):
        new_array = []
        for word in self.dictionary:
            new_array.append(word[0])
        self.dictionary = new_array
    
    @property
    def dictionary(self):
        return self._dictionary

    @dictionary.setter
    def dictionary(self, value):
        self._dictionary = value

        # import DictionaryProcessor

# Text Processing Method With Class
>Use this method to get **Combined Array with custom Tagger** based on **Given Dictionary**

- **Dictionary** - One dimensional string List
- **TaggedArray** - Hasil Tagged and Predicted POS Tagger yang sudah dikonversi menjadi List
- **NewLabel** - Label/Custom tagger yang ingin diberikan pada output berdasarkan dictionary

In [6]:
class TextProcessor():
    def __init__(self):
        pass
    
    def text_processing(self, dictionary, taggedArray, newLabel):
        tagging_arr = taggedArray
        

        sentence  = []
        for tag in tagging_arr:
            sentence.append(tag[0])

        location_arr = dictionary

        locationArr_index = 0
        location_index = 0
        sentence_index = 0
        tagging_index = 0
        success_count = 0
        check = []
        while_count = 0

        for word in sentence:
            for location in location_arr:
                if word == location[0]:
                    checking_arr = []
                    checking_arr.append(word)
                    for _ in range(len(location)-1):
                        sentence_index += 1
                        checking_arr.append(sentence[sentence_index])
                    sentence_index -= (len(location)-1)

                    if checking_arr == location:
                        for _ in range(len(location) - 1):
                            tagging_arr[sentence_index][0] = tagging_arr[sentence_index][0] + ' ' +  tagging_arr[tagging_index + 1][0]
                            tagging_arr[tagging_index + 1][0] = ''
                            tagging_index += 1
                        tagging_arr[sentence_index][1] = newLabel
                        tagging_index -= len(location) - 1
            sentence_index += 1
            tagging_index += 1


        new_arr = []

        for tag in tagging_arr:
            if tag[0] != '':
                new_arr.append(tag)

        return new_arr

## Extracting Date using Regex

In [7]:
import re

class DateRegex:
    _month_list = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'januari', 'februari', 'maret', 'april', 'mei', 'juni', 'juli', 'agustus', 'september', 'oktober', 'november', 'desember', 'jan', 'feb', 'mar', 'apr', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec']
    
    def __init__(self, textInput: str):
        self.textInput = textInput
        self._final_date_arr = []
        
    def __threeDate(self):
        three_date = re.findall(r'\d+\/\d+\/\d+|\d+\-\d+\-\d+', self.textInput)
        for date in three_date:
            self._final_date_arr.append(date)
        regex = r'\d+\/\d+\/\d+|\d+\-\d+\-\d+'
        self.textInput = re.sub(regex, '', self.textInput)
    
    def __twoDate(self):
        test_arr = re.findall(r'\d+\-\d+|\d+\/\d+', self.textInput)
        for date in test_arr:
          self._final_date_arr.append(date)
    
    def __dateMonthYear(self):
        date_month_year = re.findall(r'\d+\s\w+\s\d+', self.textInput)
        newtag_arr = []
        for word in date_month_year:
            wordsplit = word.split()
            if wordsplit[1] in self._month_list:
                wordjoin = " ".join(wordsplit)
                newtag_arr.append(wordjoin)
        for date in newtag_arr:
          self._final_date_arr.append(date)
        regex = '\d+\s\w+\s\d+'
        self.textInput = re.sub(regex, '', self.textInput)
        
        
    def __monthDateYear(self):
        month_date_year = re.findall(r'\w+\s\d+\s\d+', self.textInput)
        newtag_arr = []
        for word in month_date_year:
            wordsplit = word.split()
            if wordsplit[1] in self._month_list:
                wordjoin = " ".join(wordsplit)
                newtag_arr.append(wordjoin)
        for date in newtag_arr:
          self._final_date_arr.append(date)
        regex = '\w+\s\d+\s\d+'
        self.textInput = re.sub(regex, '', self.textInput)
    
    def __monthDate(self):
        month_date = re.findall(r'\w+\s\d+', self.textInput)
        newtag_arr = []
        for word in month_date:
            wordsplit = word.split()
            if wordsplit[0] in self._month_list:
                wordjoin = " ".join(wordsplit)
                newtag_arr.append(wordjoin)
        for date in newtag_arr:
          self._final_date_arr.append(date)
        regex = '\w+\s\d+'
        self.textInput = re.sub(regex, '', self.textInput)
        
    def __dateMonth(self):
        tag_arr2 = (re.findall(r'\d+\s\w+', self.textInput))
        newtag_arr2 = []
        for word in tag_arr2:
            wordsplit = word.split()
            if wordsplit[1] in self._month_list:
                wordjoin = " ".join(wordsplit)
                newtag_arr2.append(wordjoin)
        for date in newtag_arr2:
          self._final_date_arr.append(date)
        
    def getDate(self):
        self.__threeDate()
        self.__twoDate()
        self.__dateMonthYear()
        self.__monthDateYear()
        self.__monthDate()
        self.__dateMonth()
        return self._final_date_arr
        

## Import Scrapped Data

In [8]:
from IPython.display import display
import pandas as pd

df = pd.read_csv('data.csv')

print(df)

                                                title  \
0   Monitoring Cagar Alam Laut Kepulauan Karimata ...   
1   Bunuh-Cincang Penyu Langka Dilindungi, 7 Nelay...   
2   Ratusan Tukik Dilepas-liarkan ke Habitatnya di...   
3   Kapolres Badung Melepas Ratusan Tukik Dan Berb...   
4   Jaga Keseimbangan Alam, Ratusan Tukik Dilepasl...   
5   Lestarikan Satwa Laut, Kapolres Badung Melepas...   
6   Melihat Lebih Dekat Pulau Sangalaki, Surga bag...   
7   KKP Gagalkan Penangkapan Penyu Hijau Ilegal di...   
8   200 Tukik Dilepasliarkan di Pantai Kuranji Ban...   
9   Tanggapi Video Pemancingan Penyu di Gunung Kid...   
10  Inilah Rencana Aksi Konservasi Perairan dan Pe...   
11  Pertama Kali dalam 13 Tahun, Penyu Belimbing K...   
12  Kisah Para Penjaga Pulau Sangalaki, Selamatkan...   
13  Empat Penyu Hijau Hasil Sitaan Jalani Rehabili...   
14  Identifikasi Ancaman Pembiakan Penyu Hijau dan...   
15  Dua Kasus Penangkapan Penyu di NTT dalam Sepek...   
16  Lihat Warga Pancing Penyu D

In [9]:
df

,title,byline,dir,content,textContent,length,excerpt,siteName,url
0,Monitoring Cagar Alam Laut Kepulauan Karimata ...,Ferdinand Waskita Suryacahya,NaN,"<div id=""readability-page-1"" class=""page""><div...",\n\t\t\t\t\t\t\t\t\n\t\t\t\t \n\t\t\tTRIBUNJAK...,1955,KEberadaan jenis satwa di perairan CAL Kepulau...,Tribunjakarta.com,https://jakarta.tribunnews.com/2021/10/05/moni...
1,"Bunuh-Cincang Penyu Langka Dilindungi, 7 Nelay...",Jauh Hari Wawan S.,NaN,"<div id=""readability-page-1"" class=""page""><div...",\n\n Sleman - Ditpolairud P...,2798,Ditpolairud Polda DIY mengamankan tujuh orang ...,detiknews,https://news.detik.com/berita-jawa-tengah/d-55...
2,Ratusan Tukik Dilepas-liarkan ke Habitatnya di...,"17/10/2021, 9:12 PM",NaN,"<div id=""readability-page-1"" class=""page""><div...","MANGUPURA, balipuspanews.com – Kapolres Badung...",1933,"MANGUPURA, balipuspanews.com – Kapolres Badung...",Balipuspanews.com,https://www.balipuspanews.com/ratusan-tukik-di...
3,Kapolres Badung Melepas Ratusan Tukik Dan Berb...,Posted by\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\...,NaN,"<div id=""readability-page-1"" class=""page""><art...",\n\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\n\t\t\t\t\t\t...,2180,Kepala Kepolisian Resor Badung AKBP Leo Dedy D...,DIVISI HUMAS POLRI,https://humas.polri.go.id/2021/09/30/kapolres-...
4,"Jaga Keseimbangan Alam, Ratusan Tukik Dilepasl...",NaN,NaN,"<div id=""readability-page-1"" class=""page""><div...",Kapolres Badung AKBP Leo Dedy Defretes melepas...,1551,"MANGUPURA, BALIPOST.com - Sebagai wujud polri ...","Jaga Keseimbangan Alam, Ratusan Tukik Dilepasl...",https://www.balipost.com/news/2021/09/24/21727...
5,"Lestarikan Satwa Laut, Kapolres Badung Melepas...",I Komang Agus Aryanta,NaN,"<div id=""readability-page-1"" class=""page""><div...","\n\t\t\t\t \n\t\t\tTRIBUN-BALI.COM, MANGUPURA ...",1906,Dalam menjaga satwa laut seperti penyu agar ti...,Tribun-bali.com,https://bali.tribunnews.com/2021/09/24/lestari...
6,"Melihat Lebih Dekat Pulau Sangalaki, Surga bag...",Kompas Cyber Media,NaN,"<div id=""readability-page-1"" class=""page""><div...","SAMARINDA, KOMPAS.com - Pulau Sangalaki merupa...",11603,Selain terumbu karang bawah laut yang diidolak...,KOMPAS.com,https://regional.kompas.com/read/2021/06/23/12...
7,KKP Gagalkan Penangkapan Penyu Hijau Ilegal di...,NaN,NaN,"<div id=""readability-page-1"" class=""page""><div...","Nusa Tenggara Timur, Borneo24.com – Upaya pena...",2292,"Nusa Tenggara Timur, Borneo24.com – Upaya pena...",Borneo24.com - Berita Terbaru dan Terkini di K...,https://borneo24.com/headlines/kkp-gagalkan-pe...
8,200 Tukik Dilepasliarkan di Pantai Kuranji Ban...,Larasati Dyah Utami,NaN,"<div id=""readability-page-1"" class=""page""><div...",\n\n\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tLaporan ...,2954,Kegiatan pelepasliaran tukik ini dilaksanakan ...,Tribunnews,https://www.tribunnews.com/regional/2021/08/11...
9,Tanggapi Video Pemancingan Penyu di Gunung Kid...,Faryyanida Putwiliani,NaN,"<div id=""readability-page-1"" class=""page""><div...",\n\n\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTRIBUNNE...,1656,PROFAUNA sebagai organisasi yang bergerak di b...,Tribunnews,https://www.tribunnews.com/regional/2021/04/03...


## Deep Copy Data

In [10]:
selected_columns = df[['textContent']]
processed_df = selected_columns.copy()
selected_columns = copy.deepcopy(df[['textContent']])

### Removing Uncessary Word

In [11]:
processed_df['textContent'] = processed_df.textContent.str.replace('\n?' , '')
processed_df['textContent'] = processed_df.textContent.str.replace('\t?' , '')

<ipython-input-11-aa1b40c53b16>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_df['textContent'] = processed_df.textContent.str.replace('\n?' , '')
<ipython-input-11-aa1b40c53b16>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_df['textContent'] = processed_df.textContent.str.replace('\t?' , '')


In [12]:
processed_df.head()

,textContent
0,TRIBUNJAKARTA.COM - Cagar Alam Laut (CAL) Kep...
1,Sleman - Ditpolairud Polda...
2,"MANGUPURA, balipuspanews.com – Kapolres Badung..."
3,30 September 2021Kepala Kepolisian Resor Badun...
4,Kapolres Badung AKBP Leo Dedy Defretes melepas...


In [13]:
processed_df["textContent_without_stopwords"] = ""
processed_df.head()

,textContent,textContent_without_stopwords
0,TRIBUNJAKARTA.COM - Cagar Alam Laut (CAL) Kep...,
1,Sleman - Ditpolairud Polda...,
2,"MANGUPURA, balipuspanews.com – Kapolres Badung...",
3,30 September 2021Kepala Kepolisian Resor Badun...,
4,Kapolres Badung AKBP Leo Dedy Defretes melepas...,


## Removing Stopword

In [14]:
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
processed_df['textContent_without_stopwords'] = processed_df['textContent'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
processed_df.head()

,textContent,textContent_without_stopwords
0,TRIBUNJAKARTA.COM - Cagar Alam Laut (CAL) Kep...,TRIBUNJAKARTA.COM - Cagar Alam Laut (CAL) Kepu...
1,Sleman - Ditpolairud Polda...,Sleman - Ditpolairud Polda DIY mengamankan tuj...
2,"MANGUPURA, balipuspanews.com – Kapolres Badung...","MANGUPURA, balipuspanews.com – Kapolres Badung..."
3,30 September 2021Kepala Kepolisian Resor Badun...,30 September 2021Kepala Kepolisian Resor Badun...
4,Kapolres Badung AKBP Leo Dedy Defretes melepas...,Kapolres Badung AKBP Leo Dedy Defretes melepas...


## Case Lowering

In [15]:
processed_df['textContent_without_stopwords'] = processed_df['textContent_without_stopwords'].str.lower()
processed_df.head()

,textContent,textContent_without_stopwords
0,TRIBUNJAKARTA.COM - Cagar Alam Laut (CAL) Kep...,tribunjakarta.com - cagar alam laut (cal) kepu...
1,Sleman - Ditpolairud Polda...,sleman - ditpolairud polda diy mengamankan tuj...
2,"MANGUPURA, balipuspanews.com – Kapolres Badung...","mangupura, balipuspanews.com – kapolres badung..."
3,30 September 2021Kepala Kepolisian Resor Badun...,30 september 2021kepala kepolisian resor badun...
4,Kapolres Badung AKBP Leo Dedy Defretes melepas...,kapolres badung akbp leo dedy defretes melepas...


## Chunking Sentence

In [16]:
class AnimalTextChecker:
    def __init__(self, value_dictionary, newsText):
        self.value_dictionary = value_dictionary
        self.newsText = newsText
        self.filter_string = []
    
    def filter_sentence(self):
        for char in self.newsText:
            if any(map(str.isdigit, char)):
                for animal in self.value_dictionary:
                    temp_value = ' '.join(animal)
                    if temp_value in char:
                        self.filter_string.append(char)
                        break
        print(self.filter_string)

---

In [17]:
new_df = pd.DataFrame()

In [18]:
class WordExtractor:
    def __init__(self, df_source, df_output, value_dictionary, column_name):
        self.df_source = df_source
        self.df_output = df_output
        self.value_dictionary = value_dictionary
        self.column_name = column_name
        pass
    
    def get_value(self):
        for index, row in self.df_source.iterrows():
            value_box = []
            text = self.df_source.iloc[row.name]['textContent_without_stopwords']
            news_index = row.name
            for value in self.value_dictionary:
                temp_value = ' '.join(value)
                if temp_value in text:
                    #.lowernya udah dihapus
                    value_box.append(value)
            self.df_output = self.df_output.append({'news_index': news_index, self.column_name: value_box}, ignore_index=True)
        return self.df_output
    
    def get_date(self):
        for index, row in self.df_source.iterrows():
            value_box = []
            text = self.df_source.iloc[row.name]['textContent_without_stopwords']
            news_index = row.name
            regexDate = DateRegex(text)
            value_box.append(regexDate.getDate())
            self.df_output = self.df_output.append({'news_index': news_index, self.column_name: value_box}, ignore_index=True)
            value_box = []
        return self.df_output

In [19]:
extractedProvince_df = WordExtractor(processed_df,new_df, provinsi_dictionary, "provinsi").get_value()
extractedDate_df = WordExtractor(processed_df,new_df, provinsi_dictionary, "date").get_date()
extractedAnimal_df = WordExtractor(processed_df,new_df, animal_dictionary, "hewan").get_value()
extractedKabupaten_df = WordExtractor(processed_df,new_df, kabupaten_dictionary, "kabupaten").get_value()

In [20]:
df_news = processed_df
get_animal = extractedAnimal_df["hewan"]
get_prov = extractedProvince_df["provinsi"]
get_kabupaten = extractedKabupaten_df["kabupaten"]
get_date = extractedDate_df["date"]

df_news = df_news.join(get_animal)
df_news = df_news.join(get_prov)
df_news = df_news.join(get_kabupaten)
df_news = df_news.join(get_date)

display(df_news)

,textContent,textContent_without_stopwords,hewan,provinsi,kabupaten,date
0,TRIBUNJAKARTA.COM - Cagar Alam Laut (CAL) Kep...,tribunjakarta.com - cagar alam laut (cal) kepu...,"[[nila], [penyu], [teri]]","[[kalimantan barat], [jakarta]]","[[kepulauan seribu], [ketapang]]",[[5/10/2021]]
1,Sleman - Ditpolairud Polda...,sleman - ditpolairud polda diy mengamankan tuj...,"[[elang], [ikan], [penyu]]",[],"[[sleman], [serang], [serang], [belu]]","[[22/4/2021, 26 maret 2021]]"
2,"MANGUPURA, balipuspanews.com – Kapolres Badung...","mangupura, balipuspanews.com – kapolres badung...","[[ikan], [katak], [penyu]]",[[bali]],"[[bangka], [badung], [denpasar]]",[[24/9/2021]]
3,30 September 2021Kepala Kepolisian Resor Badun...,30 september 2021kepala kepolisian resor badun...,"[[ikan], [penyu]]",[[bali]],"[[bangka], [badung]]","[[24/9, 30 september 2021]]"
4,Kapolres Badung AKBP Leo Dedy Defretes melepas...,kapolres badung akbp leo dedy defretes melepas...,"[[ikan], [penyu]]",[[bali]],[[badung]],[[24/9]]
5,"TRIBUN-BALI.COM, MANGUPURA - Dalam menjaga sa...","tribun-bali.com, mangupura - dalam menjaga sat...","[[ikan], [penyu]]",[[bali]],"[[bangka], [badung], [gianyar]]",[[24 september 2021]]
6,"SAMARINDA, KOMPAS.com - Pulau Sangalaki merupa...","samarinda, kompas.com - pulau sangalaki sekian...","[[ayam], [beruk], [beruk], [dara], [ikan], [ke...","[[bali], [kalimantan timur]]","[[batu bara], [agam], [bangka], [cilacap], [pa...","[[18/6/2021, 17/6/2021, 17/6/2021, 17/6/2021, ..."
7,"Nusa Tenggara Timur, Borneo24.com – Upaya pena...","nusa tenggara timur, borneo24.com – upaya pena...","[[elang], [ikan], [penyu], [teri]]",[[nusa tenggara timur]],"[[kupang], [sabu raijua], [kupang]]",[[106/2018]]
8,"Laporan Wartawan Tribunnews, Larasati Dyah Uta...","laporan wartawan tribunnews, larasati dyah uta...","[[angsa], [anjing], [dara], [ikan], [kera], [n...",[[nusa tenggara barat]],"[[lombok barat], [buru]]","[[10/8/2021, 1/12/2018]]"
9,TRIBUNNEWS.COM - PROFAUNA sebagai organisasi y...,tribunnews.com - profauna organisasi bergerak ...,"[[elang], [ikan], [panda], [penyu]]",[[yogyakarta]],"[[pati], [gunung kidul], [yogyakarta], [belu]]","[[2/4/2021, 27/3/2021]]"
